<a href="https://colab.research.google.com/github/Noelleip/Module1_Project/blob/main/defaultmodel_2_11_1st_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
import os
import pathlib
import cv2
import imghdr
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 64
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 160
MODEL_NAME="model"
UNET="unet_model"
MOBILE_NET_V2="mobilenet_v2"
LEARNING_RATE=0.01
INITIAL_EPOCHS = 100

In [20]:
data_dir ="datasets"
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))

In [21]:
class_names=list(data_dir.glob('*/'))
print(class_names)

[PosixPath('datasets/9_barbecue'), PosixPath('datasets/3_egg_puff'), PosixPath('datasets/5_glutinous'), PosixPath('datasets/10_stinky'), PosixPath('datasets/8_taro'), PosixPath('datasets/6_water_chestnut'), PosixPath('datasets/.ipynb_checkpoints'), PosixPath('datasets/2_sticky'), PosixPath('datasets/1_egg_tarts'), PosixPath('datasets/11_spring_roll'), PosixPath('datasets/7_sampan'), PosixPath('datasets/4_soy_sauce')]


In [22]:
def prepare():
    global train_ds
    global val_ds

    # Loading dataset from folders
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=BATCH_SIZE)
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=BATCH_SIZE)

    # Configure the dataset for performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return

In [23]:
def train():
    global model
    global train_ds
    global val_ds

    prepare()

    # Data augmentation
    data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal_and_vertical",
                        input_shape=(IMAGE_HEIGHT,
                                    IMAGE_WIDTH,
                                    3)),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.05,0.05),
        tf.keras.layers.RandomBrightness(0.2)
    ]
    )

    # Preparing model
    num_classes = len(class_names)
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = tf.keras.Sequential([
        data_augmentation,# Data augmentation Layer
            tf.keras.layers.Rescaling(1./255),# Standardize the data (0~1)
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(180, activation='relu'),
            tf.keras.layers.Dense(num_classes)
        ])
        base_learning_rate = 0.0001
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
        model.summary()
        history=model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100
        )
        model.save("sample_data")
    return

In [24]:
if __name__ == '__main__':
    train()

Found 102 files belonging to 12 classes.
Using 82 files for training.
Found 102 files belonging to 12 classes.


Using 20 files for validation.
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (None, 160, 160, 3)       0         
                                                                 
 rescaling_4 (Rescaling)     (None, 160, 160, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 158, 158, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 79, 79, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 77, 77, 32)        9248      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 38, 38, 32)       0         
 g2D)                  

2/2 [==============================] - 19s 4s/step - loss: 2.5353 - accuracy: 0.0488 - val_loss: 2.6267 - val_accuracy: 0.1000
Epoch 2/100
2/2 [==============================] - 3s 3s/step - loss: 2.4505 - accuracy: 0.0976 - val_loss: 2.4941 - val_accuracy: 0.0500
Epoch 3/100
2/2 [==============================] - 3s 1s/step - loss: 2.4116 - accuracy: 0.1098 - val_loss: 2.4280 - val_accuracy: 0.1000
Epoch 4/100
2/2 [==============================] - 5s 4s/step - loss: 2.3885 - accuracy: 0.1585 - val_loss: 2.4244 - val_accuracy: 0.2500
Epoch 5/100
2/2 [==============================] - 3s 1s/step - loss: 2.3704 - accuracy: 0.1829 - val_loss: 2.4309 - val_accuracy: 0.2000
Epoch 6/100
2/2 [==============================] - 4s 1s/step - loss: 2.3688 - accuracy: 0.1463 - val_loss: 2.4354 - val_accuracy: 0.1000
Epoch 7/100
2/2 [==============================] - 3s 1s/step - loss: 2.3749 - accuracy: 0.1463 - val_loss: 2.4006 - val_accuracy: 0.0500
Epoch 8/100
2/2 [============================